# Финальный проект

Работу выполнила Камская Милена БКЛ-211

Основные функции бота:
1. Генерация текста на цепях Маркова
2. Поиск наиболее подходящего ответа из базы данных
3. Поиграться с моделями(найти максимально похожее слово, выбросить лишнее из строки), обученной на выкачаных и обработаных текстах

Все нужные импорты для первой части будут здесь:

In [1]:
import string
from string import punctuation
import re
import pymorphy2
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Имя\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Для начала собрем все книги, на основе которых мы будем обучать модель. В списке lst лежат ссылки на 30 рандомных книг, написанных в жанре фантастика русскими авторами. С помощью BeautifulSoup мы выкачиваем текст со всех страниц и складываем его в один большой файл формата .txt, с которым и будем работать в дальнейшем. Вышло примерно 130к строк

In [27]:
import requests
from bs4 import BeautifulSoup
import re

def get_text(url):
    rs = requests.get(url).text
    text = re.sub(r'\<[^>]*\>', '', rs)

    return text

with open('sbornik.txt', 'a', encoding='UTF-8') as output:
    lst = ['http://lib.ru/RUFANT/ABRANOW/horsmen1.txt', 'http://lib.ru/RUFANT/ABRANOW/horsmen2.txt', 'http://lib.ru/RUFANT/ABRANOW/horsmen3.txt', 
           'http://lib.ru/RUFANT/ABRANOW/selesta.txt', 'http://lib.ru/RUFANT/ABRANOW/aladdine.txt', 'http://lib.ru/RUFANT/ABRANOW/goodnght.txt', 
           'http://lib.ru/RUFANT/ABRANOW/guest.txt', 'http://lib.ru/RUFANT/ABRANOW/imprshad.txt', 'http://lib.ru/RUFANT/ABRANOW/permit.txt', 
           'http://lib.ru/RUFANT/ABRANOW/prince.txt', 'http://lib.ru/RUFANT/ABRANOW/timescal.txt', 'http://lib.ru/RUFANT/ABRANOW/walking.txt', 
           'http://lib.ru/RUFANT/ABRANOW/r_bal.txt', 'http://lib.ru/RUFANT/ANDERSEN/deti.txt', 'http://lib.ru/RUFANT/ANDERSEN/deti2.txt', 
           'http://lib.ru/RUFANT/ANDERSEN/deti3.txt', 'http://lib.ru/RUFANT/ANDERSEN/deti4.txt', 'http://lib.ru/RUFANT/ANDERSEN/deti5.txt', 
           'http://lib.ru/NEWPROZA/WULF/asylum.txt', 'http://lib.ru/NEWPROZA/WULF/zion.txt', 'http://lib.ru/NEWPROZA/WULF/erez2.txt', 
           'http://lib.ru/NEWPROZA/WULF/vodolazia.txt', 'http://lib.ru/RUFANT/LAPIN/children.txt', 'http://lib.ru/RUFANT/LAPIN/day_13.txt',
           'http://lib.ru/RUFANT/LAPIN/magger.txt', 'http://lib.ru/KLYUCHWSKIJ/cwetok.txt', 'http://lib.ru/RUFANT/SAMSONOV/korabl.txt', 
           'http://lib.ru/RUFANT/SAMSONOV/imperia.txt', 'http://lib.ru/RUFANT/SAMSONOV/maxim.txt', 'http://lib.ru/RUFANT/SAMSONOV/r_glagol.txt']
    for item in lst:
        text = get_text(item)
        print(text + '\n', file=output)

Займемся подготовкой текста для обучения модели!

In [13]:
def tokenize_ru(file_text):
    # firstly let's apply nltk tokenization
    tokens = word_tokenize(file_text)

    # let's delete punctuation symbols
    tokens = [i for i in tokens if (i not in string.punctuation)]

    # deleting stop_words
    stop_words = stopwords.words('russian')
    stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', '...'])
    tokens = [i for i in tokens if (i not in stop_words)]

    # cleaning words
    tokens = [i.replace("«", "").replace("»", "") for i in tokens]

    return tokens

In [16]:
sbornik = open('sbornik.txt', 'r', encoding='utf-8').read()

In [17]:
sentences = [tokenize_ru(sent) for sent in sent_tokenize(sbornik, 'russian')]

In [18]:
print(len(sentences))

199855


Используем морфологический парсинг!

In [26]:
from pymorphy2 import MorphAnalyzer

morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    res = list()
    for item in text:
        p = morph.parse(item)[0]
        res.append(p.normal_form)

    return res


In [27]:
with open('sbornik.txt', 'w', encoding='UTF-8') as output:
    for sent in sentences:
        lem = lemmatize(sent)
        lem_sentence = ' '.join(lem)
        print(lem_sentence, file=output)

In [1]:
import string

s = open('sbornik.txt', 'r', encoding='UTF-8').read()
s_del = ''.join(c for c in s if c not in string.ascii_letters)
s_r = s_del.replace('-', '')
with open('sbornik.txt', 'w', encoding='UTF-8') as output:
    print(s_r, file=output)

Поздравляю, мы подготовили тексты к следующему этапу!

### Приступаем к обучению модели!

In [1]:
import gensim
import logging
import urllib.request
import warnings
warnings.filterwarnings('ignore')

In [2]:
f = 'sbornik.txt'
data = gensim.models.word2vec.LineSentence(f)

In [3]:
model = gensim.models.Word2Vec(data, vector_size=300, window=5, min_count=2, epochs=50)

Здесь я обучила модель, теперь посмотрим сколько слов в словаре

In [4]:
print(len(model.wv.key_to_index))

32586


Далее я проверяю работу функций. Все стандартно

In [5]:
def most_similar_one(word):
    return model.wv.most_similar(str(word).lower(), topn=10)

most_similar_one('Мама')

[('папа', 0.4027484357357025),
 ('таня', 0.36235761642456055),
 ('феликс', 0.35526520013809204),
 ('танечка', 0.3332690894603729),
 ('тамара', 0.3192366361618042),
 ('подруга', 0.31833958625793457),
 ('минни', 0.31569305062294006),
 ('коля', 0.3117457330226898),
 ('дядя', 0.3073084056377411),
 ('барышня', 0.3023766279220581)]

In [6]:
def most_similar_two(word):
    return model.wv.most_similar(positive=str(word.split()[0]).lower(), negative=str(word.split()[1]).lower(), topn=10)

most_similar_two('хороший плохой')

[('банкир', 0.21765203773975372),
 ('целый', 0.20187951624393463),
 ('строго', 0.20174622535705566),
 ('шура', 0.19786711037158966),
 ('блёкло', 0.19669906795024872),
 ('затормозить', 0.19427995383739471),
 ('пустовать', 0.1918061524629593),
 ('педантично', 0.19146472215652466),
 ('чум', 0.1912911832332611),
 ('поролоновомебельный', 0.19083306193351746)]

In [7]:
def doesnt_match(line):
    return model.wv.doesnt_match(line.lower().split())

doesnt_match('Родитель семья сын дочь')

'дочь'

## Разбираемся с поиском ответов по базе!

Я нашла какую-то базу, которую предлагали на олимпиаде по машинному обучению. В ней лежат ответы на вопросы с пометкой good/bad/neutral. Я заранее почистила ее от плохих слов, чтобы бот оставался на добром, но, простите, ничего не обещаю... просто выключайте его, переходите к другим функциям - не терпите. 

In [8]:
import pandas as pd
good = pd.read_csv('good.tsv', sep='\t', on_bad_lines='skip')
good.sample(3)

,context_2,context_1,context_0,reply,label
19639,NaN,NaN,"если ты не хочешь , чтобы она знала , я не ска...",елена ?,good
18302,теперь ты в безопасности .,"никому не нужно знать , что ты здесь .",где мой дядя ?,он мертв .,good
2150,NaN,NaN,"о , да , я могу остаться .",давай,neutral


Превращаем тексты в числовые векторы, чтобы осуществлять по ним приближённый поиск

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

vectorizer.fit(good.context_0)
matrix_big = vectorizer.transform(good.context_0)
print(matrix_big.shape)

(59977, 14115)


Сокращаем размерность матрицы. Алгоритм PCA (метод главных компонент) подбирает матрицу проекции так, чтобы исходную матрицу можно было потом восстановить с наименьшей среднеквадратической ошибкой.

Мы представляем каждый текст не 14123-мерным, а всего лишь 300-мерным вектором

In [10]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300)
svd.fit(matrix_big)
matrix_small = svd.transform(matrix_big)

print(matrix_small.shape)
print(svd.explained_variance_ratio_.sum())

(59977, 300)
0.43971940059690495


Пишем класс для случайного выбора из всех подходящих ответов 

In [11]:
import numpy as np
from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator

def softmax(x):
    proba = np.exp(-x)
    return proba / sum(proba)

class NeighborSampler(BaseEstimator):
    def init__(self, k=5, temperature=1.0):
        self.k = k
        self.temperature = temperature
    def fit(self, X, y):
        self.tree = BallTree(X)
        self.y_ = np.array(y)
    def predict(self, X, random_state=None):
        distances, indices = self.tree.query(X, return_distance=True)
        result = []
        for distance, index in zip(distances, indices):
            result.append(np.random.choice(index, p=softmax(distance)))
        return self.y_[result]

Все соединяем

In [12]:
from sklearn.pipeline import make_pipeline

ns = NeighborSampler()
ns.fit(matrix_small, good.reply)
pipe = make_pipeline(vectorizer, svd, ns)

In [13]:
def talk(text):
    return str(pipe.predict([str(text)]))[2:-2]

talk('как дела?')

'хорошо . а у вас ?'

## Генерация на цепях Маркова

Тут уже не так весело, но это достаточно простая и милая генерация, основанная на текстах, которые я выкачала. 

Цепь Маркова — это последовательность событий, где каждое новое событие зависит только от предыдущего. Например, после одного слова может стоять другое слово. Для нашей работы алгоритму всегда нужен исходный текст (он же корпус) — глядя на этот текст, алгоритм поймёт, какие слова обычно идут друг за другом.

In [14]:
import numpy as np
text = open('sbornik_old.txt', encoding='utf8').read()
corpus = text.split()

def make_pairs(corpus):
    # перебираем все слова в корпусе, кроме последнего
    for i in range(len(corpus)-1):
        # генерируем новую пару и возвращаем её как результат работы функции
        yield (corpus[i], corpus[i+1])

def markov(www):        
    pairs = make_pairs(corpus)
    word_dict = {}
    for word_1, word_2 in pairs:
        if word_1 in word_dict.keys():
            word_dict[word_1].append(word_2)
        else:
            word_dict[word_1] = [word_2]
    first_word = www
    while first_word.islower():
        first_word = np.random.choice(corpus)
    chain = [first_word]
    n_words = 10
    for i in range(n_words):
        chain.append(np.random.choice(word_dict[chain[-1]]))
    return ' '.join(chain)

## TelegramBot

Ну тут все для бота

In [27]:
%pip install pyTelegramBotAPI

Note: you may need to restart the kernel to use updated packages.


In [26]:
%pip install python-telegram-bot[socks]

Note: you may need to restart the kernel to use updated packages.


In [1]:
import telebot  # импортируем модуль pyTelegramBotAPI
import conf     # импортируем наш секретный токен
from telebot import types

bot = telebot.TeleBot(conf.TOKEN)

Далее прописываю реакцию на комманды

In [2]:
@bot.message_handler(commands=['start'])
def start(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Расскажи, что ты умеешь")
    btn2 = types.KeyboardButton("Хочу побаловаться с моделью!")
    btn3 = types.KeyboardButton("Плевать, давай генерировать!")
    markup.add(btn1, btn2, btn3)
    bot.send_message(message.chat.id, 'Здравствуйте!', reply_markup=markup)

In [18]:
@bot.message_handler(commands=['help'])
def help(message):
    helpani = '''Что я умею?

• Генерировать текст при помощи цепей Маркова
• Общаться с тобой, используя ответы из базы данных
• Выводить некоторые прикольчики из моделей - попробуй и узнаешь!'''
    bot.send_message(message.chat.id, helpani)

In [19]:
@bot.message_handler(commands=['bye'])
def finish(message):
    bot.send_message(message.chat.id, 'До скорых встреч! Целую, обнимаю)')

Вы обнаружили нечто

In [20]:
@bot.message_handler(commands=['special'])
def wow(message):
    bot.send_message(message.chat.id, 'Поздравляю, ты нашел пасхалку!')
    photo = open("пасхалка/" + 'халло.jpg', "rb")
    bot.send_photo(message.chat.id, photo, 'Так и я делала этот проект, всем спасибо за внимание!')

Теперь прописываю его реакцию на текстовые сообщения от пользователя

In [21]:
@bot.message_handler(content_types=['text'])
def func(message: types.Message):
    if message.text == "Расскажи, что ты умеешь":
        helpani = '''Что я умею?

                    • Генерировать текст при помощи цепей Маркова
                    • Общаться с тобой, используя ответы из базы данных
                    • Выводить некоторые прикольчики из моделей - попробуй и узнаешь!'''
        bot.send_message(message.chat.id, helpani)
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        button1 = types.KeyboardButton("Вернемся в главное меню!")
        markup.add(button1)
        bot.send_message(message.chat.id, 
                         text="Ну чего, возвращаемся в главное меню?", 
                         reply_markup=markup)
        
    elif message.text == "Вернемся в главное меню!":
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        btn1 = types.KeyboardButton("Расскажи, что ты умеешь")
        btn2 = types.KeyboardButton("Хочу побаловаться с моделью!")
        btn3 = types.KeyboardButton("Плевать, давай генерировать!")
        markup.add(btn1, btn2, btn3)
        bot.send_message(message.chat.id,
                         text="С возвращением, друг!",
                         reply_markup=markup)

    elif message.text == "Хочу побаловаться с моделью!":
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        m_sim = types.KeyboardButton("Хочу найти ближайшее слово к моему слову!")
        a_sim = types.KeyboardButton("Хочу найти ассоциацию к словам!")
        d_match = types.KeyboardButton("Хочу найти лишнее слово в строке")
        b = types.KeyboardButton("Вернемся в главное меню!")
        markup.add(m_sim, a_sim, d_match)
        markup.row(b)
        bot.send_message(message.chat.id,
                         text="Выбери действие, друг!",
                         reply_markup=markup)
        
    elif message.text == "Хочу найти ближайшее слово к моему слову!":
        bot.send_message(message.chat.id, text="Отправь мне слово") 
        bot.register_next_step_handler(message, msimilar_one)
    
    elif message.text == "Хочу найти ассоциацию к словам!":
        bot.send_message(message.chat.id, text="Отправь мне ровно 2 слова с разными оттенками значения(сначала positive, затем negаtive) через пробел") 
        bot.register_next_step_handler(message, msimilar_two)

    elif message.text == "Хочу найти лишнее слово в строке":
        bot.send_message(message.chat.id, text="Отправь мне четыре слова через пробел") 
        bot.register_next_step_handler(message, match_line)

    elif message.text == "Плевать, давай генерировать!":
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        first_option = types.KeyboardButton("Сгенерируй мне что-то, в чем я буду вечно искать смысл")
        second_option = types.KeyboardButton("А болтать ты умеешь?")
        b = types.KeyboardButton("Вернемся в главное меню!")
        markup.add(first_option, second_option)
        markup.row(b)
        bot.send_message(message.chat.id,
                         text="Выбери действие, друг!",
                         reply_markup=markup)
    
    elif message.text == "Сгенерируй мне что-то, в чем я буду вечно искать смысл":
        bot.send_message(message.chat.id, text="Отправь мне слово") 
        bot.register_next_step_handler(message, gen_markov)
    
    elif message.text == "А болтать ты умеешь?":
        bot.send_message(message.chat.id, text="Просто начни диалог)")
        bot.register_next_step_handler(message, boltalka)
        
    else:
        bot.send_message(message.chat.id, boltalka(message))      
    
def msimilar_one(message):
    try: 
        text = most_similar_one(message.text)
    except KeyError:
        text = 'Прости, я таких слов не знаю. Попробуй что-то другое'
    bot.send_message(message.chat.id, text)

def msimilar_two(message):
    try: 
        text = most_similar_two(message.text)
    except KeyError:
        text = 'Прости, я таких слов не знаю. Попробуй что-то другое'
    bot.send_message(message.chat.id, text)

def match_line(message):
    try: 
        text = doesnt_match(message.text)
    except KeyError:
        text = 'Прости, я таких слов не знаю. Попробуй что-то другое'
    bot.send_message(message.chat.id, text)

def gen_markov(message):
    try: 
        text = markov(message.text)
    except KeyError:
        text = 'Прости, я таких слов не знаю. Попробуй что-то другое'
    bot.send_message(message.chat.id, text)

def boltalka(message):
    text = talk(message.text)
    bot.send_message(message.chat.id, text)

И запускаю

In [3]:
bot.polling(none_stop=True, interval=0)

## Итоги

Этот проект не был нацелен на создание идеального бота, который сгенерирует вам осмысленный текст, с помощью нейронок от TensorFlow, к примеру.

Основная цель этого проекта - собрав тексты и обработав их, показать работу обученной модели, а так же представить генерацию текста цепями Маркова (и запихать это все в бота для удобства пользования). Здесь четко видно, где еще можно развиваться (к примеру, увеличить библиотеку... раз в 100, что поможет нам добиться большей "осмысленности"). Однако, свое творения я люблю даже таким немного глупеньким!

Большое спасибо за внимание!